# Introduction to Cross-Validation - Lab

## Introduction

In this lab, you'll be able to practice your cross-validation skills!


## Objectives

You will be able to:

- Perform cross validation on a model to determine optimal model performance
- Compare training and testing errors to determine if model is over or underfitting

## Let's get started

We included the code to pre-process below.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

ames = pd.read_csv('ames.csv')

continuous = ['LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']
categoricals = ['BldgType', 'KitchenQual', 'SaleType', 'MSZoning', 'Street', 'Neighborhood']

ames_cont = ames[continuous]

# log features
log_names = [f'{column}_log' for column in ames_cont.columns]

ames_log = np.log(ames_cont)
ames_log.columns = log_names

# normalize (subract mean and divide by std)

def normalize(feature):
    return (feature - feature.mean()) / feature.std()

ames_log_norm = ames_log.apply(normalize)

# one hot encode categoricals
ames_ohe = pd.get_dummies(ames[categoricals], prefix=categoricals, drop_first=True)

preprocessed = pd.concat([ames_log_norm, ames_ohe], axis=1)

X = preprocessed.drop('SalePrice_log', axis=1)
y = preprocessed['SalePrice_log']

### Train-test split

Perform a train-test split with a test set of 20%.

In [3]:
from sklearn.model_selection import train_test_split

In [4]:
# Split the data into training and test sets (assign 20% to test set)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [5]:
# A brief preview of train-test split
print(len(X_train), len(X_test), len(y_train), len(y_test))


1168 292 1168 292


### Fit the model

Fit a linear regression model and apply the model to make predictions on test set

In [6]:
# Your code here
from sklearn.linear_model import LinearRegression

linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_hat_train = linreg.predict(X_train)
y_hat_test = linreg.predict(X_test)

### Residuals and MSE

Calculate the residuals and the mean squared error on the test set

In [10]:
# Your code here
from sklearn.metrics import mean_squared_error

train_residuals = y_hat_train - y_train
test_residuals = y_hat_test - y_test

print(train_residuals)
print(test_residuals)

254     0.011009
1066   -0.155149
638     0.276992
799    -0.198950
380     0.501386
          ...   
1095    0.211084
1130    0.721629
1294   -0.367506
860    -0.076793
1126    0.309943
Name: SalePrice_log, Length: 1168, dtype: float64
892    -0.504106
1105   -0.016240
413    -0.162888
522    -0.239959
1036   -0.389970
          ...   
479     0.531505
1361    0.095648
802    -0.121225
651     0.377204
722    -0.259705
Name: SalePrice_log, Length: 292, dtype: float64


In [11]:
train_mse = mean_squared_error(y_hat_train, y_train)
test_mse = mean_squared_error(y_hat_test, y_test)

print(train_mse)
print(test_mse)

0.1602569596465516
0.17595331097301337


## Cross-Validation: let's build it from scratch!

### Create a cross-validation function

Write a function `kfolds()` that splits a dataset into k evenly sized pieces. If the full dataset is not divisible by k, make the first few folds one larger then later ones.

We want the folds to be a list of subsets of data!

In [15]:
def kfolds(data, k):
    # Force data as pandas DataFrame
    data = pd.DataFrame(data)
    num_observations = len(data)
    fold_size = num_observations // k
    leftovers = num_observations % k
    folds = []
    start_obs = 0
    
    for fold_n in range(1, k+1):
        if fold_n <= leftovers:
            # add 1 to fold size to account for leftovers
            fold = data.iloc[start_obs : start_obs + fold_size + 1]
            folds.append(fold)
            start_obs += fold_size + 1
        else:
            fold = data.iloc[start_obs : start_obs + fold_size]
            folds.append(fold)
            start_obs += fold_size
    
    return folds

### Apply it to the Ames Housing data

In [16]:
# Make sure to concatenate the data again
ames_data = pd.concat([X.reset_index(drop = True), y], axis = 1)

In [17]:
# Apply kfolds() to ames_data with 5 folds
ames_folds = kfolds(ames_data, 5)

### Perform a linear regression for each fold and calculate the training and test error

Perform linear regression on each and calculate the training and test error: 

In [18]:
test_errs = []
train_errs = []
k=5

for n in range(k):
    
    # Split in train and test for the fold
    train = pd.concat([fold for i, fold in enumerate(ames_folds) if i != n])
    test = ames_folds[n]
    
    # Fit a linear regression model
    linreg.fit(X_train, y_train)
    
    # Evaluate Train and Test errors
    y_hat_train = linreg.predict(X_train)
    y_hat_test = linreg.predict(X_test)
    train_residuals = y_hat_train - y_train
    test_residuals = y_hat_test - y_test
    train_errs.append(np.mean(train_residuals.astype(float)**2))
    test_errs.append(np.mean(test_residuals.astype(float)**2))

print(train_errs)
print(test_errs)

[0.16025695964655146, 0.16025695964655146, 0.16025695964655146, 0.16025695964655146, 0.16025695964655146]
[0.17595331097301337, 0.17595331097301337, 0.17595331097301337, 0.17595331097301337, 0.17595331097301337]


## Cross-Validation using Scikit-Learn

This was a bit of work! Now, let's perform 5-fold cross-validation to get the mean squared error through scikit-learn. Let's have a look at the five individual MSEs and explain what's going on.

In [19]:
# Your code here
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import cross_val_score

In [20]:
mse = make_scorer(mean_squared_error)

cv_5_result = cross_val_score(linreg, X, y, cv = 5, scoring = mse)

Next, calculate the mean of the MSE over the 5 cross-validation and compare and contrast with the result from the train-test split case.

In [21]:
# Your code here
cv_5_result.mean()

0.17702834210001092

##  Summary 

Congratulations! You are now familiar with cross-validation and know how to use `cross_val_score()`. Remember that the results obtained from cross-validation are robust and always use it whenever possible! 